# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


## Importación de módulos

In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues, Alarm
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger, disable_logger, logger

from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from pprint import pformat

from utils import *

set_log_level(LogLevels.INFO)
# disable_logger()
enable_logger()

## Parte 1: 

### 1. Implementación básica del problema

- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.

Para que los generales leales acuerden una decisión común, es que se usa el protocolo recursivo Oral Messages implementado en el siguiente bloque de código. Sin embargo, esa esa una especificación del comportamiento que podrían tener los generales del problema.

Es claro que vamos a tener un nodo por cada general. Además, como cada general puede enviarle un mensaje a cualquier otro general del asedio, se cumplen las siguientes restricciones:
- Bidirectional links (pues cada general puede enviarle un mensaje a cualquier otro general)
- Grafo completo (y por lo tanto conectividad)

Además, simulamos a los traidores como nodos elegidos aleatoriamente que están en un estado TRAITOR, y tienen un comportamiento dado (configurable) que continuará funcionando hasta que el asedio se de por finalizado.

Una primera implementación natural sería que que cada general cumpla el siguiente proocedimiento:
- Tomar una opinión personal: atacar la ciudad o retirarse
- Enviar a cada otro general su opinión
- Cuando haya recibido la opinión de todos los generales

TODO cuando funciona? releer biblio 1


### 2. Protocolos de comunicación


- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).

El Byzantine General Problem es una formulación específica del un problema arbitrario de generales y traidores bizantinos, que establece condiciones concretas de consistencia IC1 e IC2 disponibles en la bibliografía.
Oral Messages (OM) es un algoritmo recursivo que resuelve el BGP apoyandose en una serie de restricciones (A1, A2 y A3) para su correctitud.

Las condiciones de Oral Messages implican que agreguemos las siguientes restricciones
- Initial Distinct Values: por 3.A2
- TotalReliability por 3.A1. Sin embargo, los mensajes pueden tener demoras arbitrarias, y un nodo no puede saber si un mensaje esperado no le llega porque el otro nodo es un traidor, o porque está demorado

In [ ]:
class ByzantineGenerals(NodeAlgorithm):
    def __init__(self, simulation, *args, f=None, verbose=False, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.f = f
        self.siege = Siege(len(self.network.nodes()) - f)
        self.messages_counter = 0
        self.logging = []
        self.max_messages = self.network.size() ** (f + 2)

    default_params = {
        "Value" : "Value",
    }

    class Status(StatusValues):
        COMMANDER = "COMMANDER"
        GENERAL = "GENERAL"
        DONE = "DONE"
        TRAITOR = "TRAITOR"
    S_init = (Status.COMMANDER, Status.GENERAL, Status.TRAITOR)
    S_term = (Status.DONE,)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, # 3.A1 TODO no asumirlo?? ver notas
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        traidores = random.sample(list(self.network.nodes()), self.f)
        commander_general = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["values_tree"] = {}
            node.memory["decision_tree"] = {}
            node.memory["liutenants"] = self.network.nodes() - {node}
            node.status = self.Status.GENERAL if node not in traidores else self.Status.TRAITOR
        if commander_general not in traidores:
            commander_general.status = self.Status.COMMANDER
        commander_general.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=commander_general))
 

    def get_included_liutenants(self, node: NodeAccess, path):
        already_visited = []
        for neigh in node.memory["liutenants"]:
            if neigh.memory["unique_value"] in path:
                already_visited.append(neigh)
        return node.memory["liutenants"] - set(already_visited)

    def OM_commander(self, node : NodeAccess, packet : Data):
        liutenants = self.get_included_liutenants(node, packet.path)
        self.logging.append(f"[General {node.memory['unique_value']}] Enviando mensaje con data {packet} por el path {packet.path} a los tenientes {liutenants}")
        send_with_check(
            node, 
            datos=packet, 
            dest=liutenants,
            msj_type=self.default_params["Value"],
            algorithm=self,
        )



    def checks(self, node: NodeAccess, message = None):
        self.logging.append("\n\n")
        if message:
            datos : Data = message.data
            self.logging.append(f"[General {node.memory['unique_value']}] Le llega el mensaje con data {datos}")                        
        if node.status == self.Status.TRAITOR:
            self.logging.append(f"[Traidor {node.memory['unique_value']}]")
        elif node.status == self.Status.COMMANDER:
            self.logging.append(f"\n\n[General {node.memory['unique_value']}] Observes they should {node.memory['values_tree'][()]}")

        if self.messages_counter > self.max_messages:
            error("Cota superior de mensajes excedida. Imprima logging para mas detalles.")

    def get_layer_decisions(self, node: NodeAccess, path):
        layer = len(path)
        m = self.f - layer + 1
        nodos = {key: value for (key, value) 
                in node.memory["decision_tree"].items() 
                if (len(key) == layer and key[:layer-1] == path[:layer-1])
                }
        ronda = nodos.values()
        awaiting_values = len(node.memory["liutenants"]) - layer + 2
        # self.logging.append(f"[General {node.memory['unique_value']}] En el path {path}, esperando {awaiting_values} valores, recibidos {len(ronda)}: {list(ronda)}")
        if len(ronda) == (awaiting_values):
            return ronda
        return []


    @Status.COMMANDER
    def spontaneously(self, node: NodeAccess, _: Message):
        decision = node.memory["values_tree"][()] = self.siege.observe(node)
        self.checks(node)
        datos = Data((node.memory["unique_value"], ), decision)
        self.OM_commander(node, datos)
        if decision == GeneralDecision.ATTACK:
            self.siege.attack()
            node.status = self.Status.DONE
        else:
            self.siege.retreat()
            node.status = self.Status.DONE


    @Status.COMMANDER
    def receiving(self, node: NodeAccess, message: Message):
        error("COMMANDER::receiving", message)


    @Status.TRAITOR
    def spontaneously(self, node: NodeAccess, _: Message):
        self.checks(node)
        datos = Data((node.memory["unique_value"], ), GeneralDecision.RETREAT)
        TraitorActions.send_confusing_signal(node, self, datos)
        node.status = self.Status.DONE

    @Status.TRAITOR
    def receiving(self, node: NodeAccess, message: Message):
        self.checks(node, message)
        if self.siege.state != Siege.State.ONGOING:
            node.status = self.Status.DONE
            return
        #Por ahora solo envia, no le importa nada del tamaño del path ni nada
        datos : Data = message.data
        new_path =  datos.path + (node.memory["unique_value"],)
        new_datos = Data(new_path, GeneralDecision.RETREAT)
        TraitorActions.send_confusing_signal(node, self, new_datos, message.source)


    @Status.GENERAL
    def receiving(self, node: NodeAccess, message: Message):
        self.checks(node, message)
        datos : Data = message.data
        path = datos.path
        m = self.f - len(path) + 1
        if m > 0:
            recursive_path = path + (node.memory["unique_value"],)
            if len(recursive_path) > self.f + 1:
                return
            node.memory["decision_tree"][recursive_path] = datos.value
            new_datos = Data(recursive_path, datos.value)
            self.OM_commander(node, new_datos)
        elif m == 0:
            node.memory["decision_tree"][path] = datos.value
            ronda = self.get_layer_decisions(node, path)
            while len(path) != 1 and len(ronda) != 0: 
                maj = majority(ronda)
                path = path[:len(path)-1]
                node.memory["decision_tree"][path] = maj
                self.logging.append(f"[General {node.memory['unique_value']}] Decision en path {path}: {maj}")
                ronda = self.get_layer_decisions(node, path)
            if len(path) == 1:
                self.logging.append(f"Arbol de decisiones del General {node.memory['unique_value']}:\n{pformat(node.memory['decision_tree'], width=80, sort_dicts=True)}")
                if maj == GeneralDecision.ATTACK:
                    self.siege.attack()
                    node.status = self.Status.DONE
                else:
                    self.siege.retreat()
                    node.status = self.Status.DONE


    @Status.DONE
    def default(self, *args, **kwargs):    
        pass

In [ ]:
its = 10

def get_no_iters(layer):
    if layer == 1:
        return 500
    elif layer == 2:
        return 200
    elif layer == 3:
        return 50
    elif layer == 4:
        return 10
    else:
        return 5

for layer in range(3,4):
    print(f"Corriendo capa {layer}")
    f = layer
    n = 3*f + 1
    its = get_no_iters(layer)
    for i in range(its):
        net_gen = NetworkGenerator(directed=False)
        net = net_gen.generate_complete_network(n)
        sim = Simulation(net, check_restrictions=True)
        sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : True}),)
        sim.reset()
        sim.run()
        alg = sim.algorithms[0]
        if alg.siege.state not in {Siege.State.SUCCESSFUL, Siege.State.ABORTED}:
            logger.info(f"Iteracion {i} falló")
            with open(f"results/f={layer}_it={i}.txt", "w") as res:
                for log in alg.logging:
                    print(log, file=res)
        else:
            logger.info(f"Iteracion {i} exitosa")
        logger.info("\n\n\n")


In [ ]:
alg = sim.algorithms[0]
with open(f"results/amano.txt", "w") as f:
    for log in alg.logging:
        print(log, file=f)

- Muestre como crece la complejidad en mensajes según la cantidad de fallos f. Evalúe el caso f = 1 y luego f = 2.

Un nodo puede ejecutar OM(m) por dos razones:
- Por ser el primer comandante, en cuyo caso será el único en ejecutar OM(m).
- Por haber recibido un mensaje OM(m+1). Esto implica que todos los demas tenientes menos el comandante deben haber recibido el mensaje OM(m+1), y por lo tanto también ejecutado OM(m). Es decir, un mensaje OM(m+1) genera
    - (n-1) ejecuciones de OM(m), en caso de que el comandante sea el comandante inicial: una por cada teniente
    - (n-2) ejecuciones de OM(m), en caso contrario: una por cada teniente, excluyendo al teniente que actuó como comandante en OM(m+1)

Por otro lado, la ejecución de OM(0) implica
- n-1 mensajes sin llamada recuriva posterior, en caso de que el comandante sea el comandante inicial.
- n-2 mensajes sin llamada recuriva posterior, en caso de que el comandante sea un teniente actuanddo como comandante en un paso recursivo.

Tenemos entonces que la complejidad sigue el siguiente conjunto de reglas recursivas 
- M[OM(f)] = (n-1) * M[OM(f-1)]
- M[OM(m)] = (n-1) * M[OM(m-1)] , para 0 < m < f
- M[OM(0)] = (n-1)              , para 0 = f
- M[OM(0)] = (n-1)              , para 0 < f


Entonces: la complejidad crece exponencialmente una vez por un factor de (n-1) en la primera ejecución del comandante inicial, y luego por un factor de (n-2):
- M[OM(m)] = (n-1) exp(n-2, m-1) , para 0 < m
- M[OM(0)] = (n-1)

Para los casos evaluados, seguimos la regla n = 3*f + 1 para determinar la cantidad de generales

    f = 1

El comandante inicial envía un mensaje a cada uno de los tres tenientes.

Luego, cada teniente recibe un mensaje de una ejecución de OM(1) y ejecuta OM(0), actuando como comandante en un subproblema donde sus generales son los otros dos tenientes del escenario, a los cuales les envía dos mensajes mas.

Con cada mensaje de OM(0) de cada uno de los otros dos tenientes, y el mensaje que recibe del comandante, hace una mayoría de entre esos tres valores, y decide.

Entonces: el comandante envía 1 mensaje por cada teniente, y cada uno de los 3 tenientes envía 2 mensajes (3*2 = 6): se intercambian un total de 9 mensajes.


    f = 2

asd

- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.


In [ ]:
#TODO

### 3. Análisis

- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

In [ ]:
def find_minimum_generals_for_consensus(f : int, its : int = None):
    net_gen = NetworkGenerator(directed=False)
    protective_max = 4*f
    n_search = f
    statistic_success = False
    while not statistic_success and n_search <= protective_max:
        n_search +=1
        statistic_success = False
        net = net_gen.generate_complete_network(n_search)
        sim = Simulation(net, check_restrictions=True)
        sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : False}),)

        for i in range(its):
            sim.reset()
            sim.run()
            alg = sim.algorithms[0]
            statistic_success &= alg.siege.state != Siege.State.FAILED
            if not statistic_success:
                break
    return n_search if statistic_success else None

fs = np.array([2, 3, 4, 5])

for fi in fs:
    n_found = find_minimum_generals_for_consensus(fi, 25*fi)
    print(f"Para f={fi} se encontró n mínimo para alcanzar consenso n = {n_found}")
    if not n_found:
        print(f"No se encontro n minimo para f = {fi} en el espacio de busqueda permitido\n")
    elif n_found >= 3*fi + 1:
        print(f"Se cumple la condicion para f = {fi}: {n_found} ≥ 3*{fi} + 1\n")
    else:
        print(f"No se cumple la condicion para f = {fi}: {n_found} < 3*{fi} + 1\n")

## Parte 2: Protocolos de Commit de Transacciones

### 1. Implementación del protocolo 2PC (Two-Phase Commit)

- Simule el proceso de coordinación entre un coordinador y varios participantes.

In [ ]:
class TwoPhaseCommit(NodeAlgorithm):
    def __init__(self, simulation, waiting_time = 15,*args, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.transaction = Transaction(len(self.network.nodes()) - 1)
        self.messages_counter = 0
        self.logging = []
        self.waiting_time = waiting_time
        
    default_params = {
        "Prepare" : "Can you commit this transaction?",
        "OK" : "OK, I can commit the transaction",
        "Commit" : "Commit the transaction",
        "Abort" : "Abort the transaction",
        "PrepareTimeout" : "Timeout waiting for participant's OKs",
    }

    class Status(StatusValues):
        COORDINATOR = "COORDINATOR"
        PARTICIPANT = "PARTICIPANT"
        SUCCESS = "SUCCESS"
        CRASHED = "CRASHED"
    S_init = (Status.PARTICIPANT, Status.COORDINATOR,)
    S_term = (Status.SUCCESS, Status.CRASHED)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        #TotalReliability, Pueden perderse mensajes
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        coordinator = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["coordinator"] = coordinator
            node.memory["crash_chance"] = CrashBehavior.crash_chance(node)
            node.status = self.Status.PARTICIPANT
        coordinator.memory["oks"] = {}
        coordinator.status = self.Status.COORDINATOR
        coordinator.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=coordinator))



    def crash(self, node : NodeAccess):
        if node.status == self.Status.COORDINATOR:
            self.logging.append(f"[Coordinator] Crashed")
            self.transaction.declare_deadlock()
        else:
            self.logging.append(f"[Node {node.memory['unique_value']}] Crashed")
            self.transaction.crash()
        node.status = self.Status.CRASHED

    def send_to_participants(self, node : NodeAccess, header: str):
        for neighbor in node.neighbors():
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.send(
                node, 
                data=None,
                destination=neighbor,
                header=header
            )

    @Status.COORDINATOR
    def spontaneously(self, node: NodeAccess, _: Message):
        self.send_to_participants(node, self.default_params["Prepare"])
        m = Message(
            destination=node.memory["coordinator"],
            header=self.default_params["PrepareTimeout"]
            )
        node.memory["PrepareAlarm"] = self.set_alarm(node, self.waiting_time, m)


    @Status.COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["OK"]:
            node.memory["oks"][message.source] = True
            if len(node.memory["oks"]) == len(node.neighbors()):
                self.disable_alarm(node.memory["PrepareAlarm"])
                self.send_to_participants(node, self.default_params["Commit"])
                node.status = self.Status.SUCCESS
        else:
            error("COMMANDER::receiving", message)
        

    @Status.COORDINATOR
    def alarm(self, node: NodeAccess, message: Message):
        self.logging.append(f"[Node {node.memory['unique_value']}] Alarm triggered: {message}")
        if message.header == self.default_params["PrepareTimeout"]:
            self.send_to_participants(node, self.default_params["Abort"])
            node.status = self.Status.SUCCESS
        else:
            error("COMMANDER::receiving", message)
        


    @Status.PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Prepare"]:
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["OK"]
            )
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        elif message.header == self.default_params["Commit"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("PARTICIPANT::receiving", message)


    @Status.SUCCESS
    def default(self, *args, **kwargs):    
        pass

    @Status.CRASHED
    def default(self, *args, **kwargs):    
        pass


In [ ]:
n = 5

net_gen = NetworkGenerator(directed=False)
net = net_gen.generate_complete_network(n)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = ((TwoPhaseCommit, {}),)
sim.reset()



In [ ]:
sim.run(1)
fig = draw.draw_current_state(sim)
fig

In [ ]:



for i in range(100):
    net_gen = NetworkGenerator(directed=False)
    net = net_gen.generate_complete_network(n)
    sim = Simulation(net, check_restrictions=True)
    sim.algorithms = ((TwoPhaseCommit, {}),)
    sim.reset()
    sim.run()
    alg = sim.algorithms[0]
    if alg.transaction.state == Transaction.TransactionState.INCONSISTENT:
        logger.info(f"Iteracion {i} perdió atomicidad")
        for log in alg.logging:
            logger.info(log)
    elif alg.transaction.state == Transaction.TransactionState.DEADLOCK:
        logger.info(f"Iteracion {i} terminó en deadlock")
        for log in alg.logging:
            logger.info(log)
    logger.info("\n")



-  Analice cómo los fallos de comunicación o de nodos afectan la atomicidad del resultado.
-  Identifique situaciones de bloqueo (blocking states).

Asumimos en nuestro escenario que un Crash implica que el nodo de puede recuperarse: esta es una condición fuerte, el algoritmo está diseñado para funcionar con esa condición. Una restricción mas laxa al suceso de crash (ejemplo luego de un tiempo se recupera) podría implicar que, al recuperarme, puedo seguir con el procedimiento que estaba haciendo (ejemplo un nodo que iba a abortar)

Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion


Definimos que los participantes siempre dicen que están para hacer la transacción. Podríamos haber implementado un sistema de votación, donde es necesaria mayoría absoluta de commit para que el coordinador indique a los participantes hacer un commit.

Es importante definir qué consideramos atomicidad en nuestro modelo. Una primera definición sería decir que un resultado es atómico si todos los nodos que no fallaron ejecutan la misma accion al finalizar el algoritmo (commit o abort). 

Sin embargo, esta condición parece poco fuerte considerando los usos prácticos de 2PC, por ejemplo en copias redundantes de una base de datos donde nos interesa confirmar la transacción solo si todas las copias la confirman. Podemos exigir entonces una condición más fuerte de atomicidad, y especificar que un resultado atómico también debe ser coherente, es decir: no solo todos los nodos que no fallan deben ejecutar la misma acción, sino que también deben hacer un commit (abort) si y solo si todos los nodos también hacen commit (abort).

Es importante hacer encuadrar esta definición pues define que sucede, por ejemplo en el siguiente escenario: ¿Qué pase si todos los participantes commitean la transacción, pero uno de ellos no recibe nunca el mensaje commit/abort? en ese caso podríamos argumentar tanto a favor de una situación de bloqueo (pues la transacción tecnicamente no se confirmó/descartó) como de una falla de atomicidad (si el mensaje era un commit entonces algunos participantes confirmaron la transacción mientras que este no, que en la práctica es lo mismo que abortar*).

Darle rigurosidad a nuestra definición de atomicidad nos permite esclarecer situaciones como esta. Siempre que se de una situación de fallo de un participante, lo que el conjunto de estos debería hacer sería abortar la acción. Por otro lado, siempre que el fallo no se de en los participantes, la acción a tomar debería ser commitear.

Por que no ponemos un timer en los participantes? Poner contraejemplo sencillo, vinculando esto de atomicidad.


Consideremos todos los fallos posibles que pueden darse en este escenario:
1. Crash del coordinador cuando comienza el protocolo
    - DEADLOCK: Ningun participante recibirá intrucciones de commit/abort.
2. Crash del coordinador mientras envía mensajes de Prepare
    - DEADLOCK: Idem caso 1
3. Crash del coordinador mientras espera que le terminen de llegar todos los mensajes de OK (o antes de que termine de esperar)
    - DEADLOCK: Idem caso 1
4. Crash del coordinador mientras envía mensajes de Commit/Abort
    - DEADLOCK: Algunos participantes (a los que no se les llegó a enviar la decisión) quedarán esperando la misma.
5. Crash de un participante antes de recibir/responder un mensaje de Prepare
    - CONTEMPLADO: El coordinador detectará la falta del OK correspondiente cuando se acabe el timer. Entonces enviará un abort a todos los participantes.
6. Crash de un participante después de haber enviado un mensaje de OK, pero antes de recibir un mensaje Commit/Abort
    - ATOMICIDAD: Si un participante falla, no podrá recuperarse para recibir un mensaje de Commit/Abort, por lo que por efecto colateral será que su copia quedará en el mismo estado, que es lo mismo que abortar. Tendremos una falta de atomicidad si el mensaje que iba a recibir el participante era commit. En caso de que fuera abort conservaremos de forma accidental la atomicidad.
7. Crash de un participante despues de haber recibido un mensaje Commit/Abort, pero antes de hacer commit/abort de la transacción
    - ATOMICIDAD: Sucede lo mismo que en el caso 6. La sutileza es que si la simulación permitiese a los nodos recuperarse de un crash y continuar con su ejecución, el nodo podría terminar de hacer commit/abort cuando se recupere.
8. Pérdida de un mensaje Prepare
    - CONTEMPLADO: Idem al caso 5.
9. Pérdida de un mensaje OK
    - CONTEMPLADO: Idem al caso 5.
10. Pérdida de un mensaje Commit/Abort
    - DEADLOCK: Al participante nunca le llegará el mensaje de Commit/Abort, y se quedará esperandolo.

Observar que, en las condiciones que definimos atomicidad y bajo la implementación de nuestro algoritmo, no tenemos problemas de atomicidad: nunca sucederá, si los mensajes no son corruptos, que a algunos nodos se les envíe commit y a otros abort. Nuestro algoritmo específicamentee indica que a todos los nodos se les manda el mismo una vez que el coordinador a tomado una decisión.

Podríamos mejorar algunos casos particulares
- (Si implementamos voto de los participantes) si un votante decide que no puede confirmar al transacción, ya sabe de antemano que tiene que abortar, aunque no le llegue ningun mensaje.

*Depende un poco de como sea el modo de operación: 
- si cuando me llega el Prepare hago la acción y despues si me llega abort doy marcha atras
- si cuando me llega el Prepare no hago nada, y cuando me llega el commit hago la acción.

En todo caso, definimos que no es lo mismo abortar la transacción que no hacer nada.

In [ ]:
#TODO implementar casos?

### 2. Extensión al protocolo 3PC (Three-Phase Commit)

- Mejore el protocolo anterior incorporando una fase intermedia para evitar bloqueos permanentes.

In [ ]:
class ThreePhaseCommit(NodeAlgorithm):
    def __init__(self, simulation, waiting_time = 15,*args, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.transaction = Transaction(len(self.network.nodes()) - 1)
        self.messages_counter = 0
        self.logging = []
        self.waiting_time = waiting_time
        
    default_params = {
        "Prepare" : "Can you commit this transaction?",
        "OK" : "OK, I can commit the transaction",
        "PreCommit" : "Get ready for commiting the transaction",
        "Ready" : "I'm ready for commiting the transaction",
        "Commit" : "Commit the transaction",
        "Abort" : "Abort the transaction",
        "PrepareTimeout" : "Timeout waiting for participant's OKs",
        "PreCommitTimeout" : "Timeout waiting for coordinator's PreCommit",
        "DoCommitTimeout" : "Timeout waiting for coordinator's DoCommit",

        #TODO implementar comunicacion vecinos
    }

    class Status(StatusValues):
        COORDINATOR = "COORDINATOR"
        PARTICIPANT = "PARTICIPANT"
        POST_PREPARE_COORDINATOR = "POST_PREPARE_COORDINATOR" 
        POST_PREPARE_PARTICIPANT = "POST_PREPARE_PARTICIPANT" 
        PRE_COMMIT_COORDINATOR = "PRE_COMMIT_COORDINATOR"
        PRE_COMMIT_PARTICIPANT = "PRE_COMMIT_PARTICIPANT"
        RECOVERING = "RECOVERING"
        SUCCESS = "SUCCESS"
        CRASHED = "CRASHED"
    S_init = (Status.PARTICIPANT, Status.COORDINATOR,)
    S_term = (Status.SUCCESS, Status.CRASHED)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, #Pueden perderse mensajes TODO sacar?
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        coordinator = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["self"] = node
            node.memory["coordinator"] = coordinator
            node.memory["crash_chance"] = CrashBehavior.crash_chance(node)
            node.status = self.Status.PARTICIPANT
        coordinator.memory["oks"] = {}
        coordinator.memory["readys"] = {}
        coordinator.status = self.Status.COORDINATOR
        coordinator.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=coordinator))



    def crash(self, node : NodeAccess):
        if node.status == self.Status.COORDINATOR:
            self.logging.append(f"[Coordinator] Crashed")
            # self.transaction.declare_deadlock()
        else:
            self.logging.append(f"[Node {node.memory['unique_value']}] Crashed")
            self.transaction.crash()
        node.status = self.Status.CRASHED

    def send_to_participants(self, node : NodeAccess, header: str):
        for neighbor in node.neighbors():
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            self.send(
                node, 
                data=None,
                destination=neighbor,
                header=header
            )

    @Status.COORDINATOR
    def spontaneously(self, node: NodeAccess, _: Message):
        m = Message(
            destination=node.memory["coordinator"],
            header=self.default_params["PrepareTimeout"]
            )
        node.memory["PrepareAlarm"] = self.set_alarm(node, self.waiting_time, m)
        self.send_to_participants(node, self.default_params["Prepare"])
        self.crash(node)
        # node.status = self.Status.POST_PREPARE_COORDINATOR



    @Status.POST_PREPARE_COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["OK"]:
            node.memory["oks"][message.source] = True
            if len(node.memory["oks"]) == len(node.neighbors()):
                self.disable_alarm(node.memory["PrepareAlarm"])
                # Asumo que los nodos solo pueden fallar al principio. Una vez que confirman que pueden hacer la transaccion, no crashean
                # Entonces: no inicio ninguna alarma de nuevo espertando los readys
                self.send_to_participants(node, self.default_params["PreCommit"])
                node.status = self.Status.PRE_COMMIT_COORDINATOR
        else:
            error("POST_PREPARE_COORDINATOR::receiving", message)
        
    @Status.POST_PREPARE_COORDINATOR
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["PrepareTimeout"]:
            self.send_to_participants(node, self.default_params["Abort"])
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_COORDINATOR::alarm", message)



    @Status.PRE_COMMIT_COORDINATOR
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Ready"]:
            node.memory["readys"][message.source] = True
            if len(node.memory["readys"]) == len(node.neighbors()):
                self.send_to_participants(node, self.default_params["Commit"])
                node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_COORDINATOR::receiving", message)



    @Status.PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["Prepare"]:
            if CrashBehavior.determine_crash(node):
                self.crash(node)
                return
            m = Message(
                destination=node.memory["self"],
                header=self.default_params["PreCommitTimeout"]
                )
            node.memory["PreCommitAlarm"] = self.set_alarm(node, 2*self.waiting_time, m)
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["OK"]
            )
            node.status = self.Status.POST_PREPARE_PARTICIPANT
        else:
            error("PARTICIPANT::receiving", message)



    @Status.POST_PREPARE_PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        self.disable_alarm(node.memory["PreCommitAlarm"])
        if message.header == self.default_params["PreCommit"]:
            m = Message(
                destination=node.memory["self"],
                header=self.default_params["DoCommitTimeout"]
                )
            node.memory["DoCommitAlarm"] = self.set_alarm(node, self.waiting_time, m)
            self.send(
                node, 
                data=None,
                destination=node.memory["coordinator"],
                header=self.default_params["Ready"]
            )
            node.status = self.Status.PRE_COMMIT_PARTICIPANT
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_PARTICIPANT::receiving", message)

    @Status.POST_PREPARE_PARTICIPANT
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["PreCommitTimeout"]:
            # m = Message(
            #     destination=node.memory["self"],
            #     header=self.default_params["NeighborsRoundTimeout"]
            #     )
            # node.memory["NeighborsRoundAlarm"] = self.set_alarm(node, self.waiting_time, m)
            # self.send_to_participants(node, self.default_params["Request"])
            # node.status = self.Status.RECOVERING
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("POST_PREPARE_PARTICIPANT::alarm", message)



    @Status.PRE_COMMIT_PARTICIPANT
    def receiving(self, node: NodeAccess, message: Message):
        #Asumo que los participantes no fallan despues de confirmar que pueden hacer la transaccion
        self.disable_alarm(node.memory["DoCommitAlarm"])
        if message.header == self.default_params["Commit"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        elif message.header == self.default_params["Abort"]:
            self.logging.append(f"[Node {node.memory['unique_value']}] Aborting transaction")
            self.transaction.abort()
            node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_PARTICIPANT::receiving", message)

    @Status.PRE_COMMIT_PARTICIPANT
    def alarm(self, node: NodeAccess, message: Message):
        if message.header == self.default_params["DoCommitTimeout"]:
            #Se que a los otros nodos tambien les debe haber llegado un precommit (asumiendo que no se pierden mensajes)
            self.logging.append(f"[Node {node.memory['unique_value']}] Committing transaction")
            self.transaction.commit()
            node.status = self.Status.SUCCESS
        else:
            error("PRE_COMMIT_PARTICIPANT::alarm", message)




    #TODO implement recovery
    @Status.RECOVERING
    def default(self, *args, **kwargs):    
        pass


    @Status.SUCCESS
    def default(self, *args, **kwargs):    
        pass

    @Status.CRASHED
    def default(self, *args, **kwargs):    
        pass


In [ ]:
plt.close()

In [ ]:
n = 5

net_gen = NetworkGenerator(directed=False)
net = net_gen.generate_complete_network(n)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = ((ThreePhaseCommit, {}),)
sim.reset()

In [ ]:
sim.run(1)
fig = draw.draw_current_state(sim)
fig

In [ ]:
n = 5

for i in range(100):
    net_gen = NetworkGenerator(directed=False)
    net = net_gen.generate_complete_network(n)
    sim = Simulation(net, check_restrictions=True)
    sim.algorithms = ((ThreePhaseCommit, {}),)
    sim.reset()
    sim.run()
    alg = sim.algorithms[0]
    if alg.transaction.state == Transaction.TransactionState.INCONSISTENT:
        logger.info(f"Iteracion {i} perdió atomicidad")
        for log in alg.logging:
            logger.info(log)
    elif alg.transaction.state == Transaction.TransactionState.DEADLOCK:
        logger.info(f"Iteracion {i} terminó en deadlock")
        for log in alg.logging:
            logger.info(log)
    logger.info("\n")



- Demuestre en qué condiciones se logra una mayor tolerancia a fallos.

Lorem ipsum

### 3. Integración con escenarios bizantinos

- Modifique el protocolo para tolerar fallos bizantinos, incorporando firmas digitales y verificación de mensajes.

In [ ]:
#TODO

- Simule casos en los que un participante miente sobre su voto o altera mensajes.

In [ ]:
 #TODO